In [1]:
# Import Relevant Libraries
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
# Reading files in folder
direc = "Master Data"
all_files = os.listdir(direc)
jig_files = [file for file in all_files if "rarm" in file]

y_train_back = [] #right side result
y_train_front = [] #left side result
x_train = []

data_types = [" fx[N]", " fy[N]", " fz[N]"] #," mx[Nm]", " my[Nm]", " mz[Nm]" excluded for now
# Extracting data from CSV and giving them labels
for file_name in jig_files:
    df = pd.read_csv(os.path.join(direc, file_name))
    if df.shape[0] > 450:
        continue
    # if df.shape[0] > 450 or df.shape[0] < 400: #For front
    #     continue
    # # print(df.head())
    data = []
    for tst in data_types:
        temp = df[tst].values
        temp = temp[0:220]
        # print(temp) 
        data.append(temp)
    x_train.append(np.concatenate(data))
    # print(x_train)
    
    back_res = file_name[-7:-6] #result of rihgt side
    front_res = file_name[-5:-4] #result of left side

    #consider imcomplete folding as Fail and missing fold as Fail as well
    if back_res == "P":
        y_train_back.append("P")
    else:
        y_train_back.append("F")
    
    if front_res == "P":
        y_train_front.append("P")
    else:
        y_train_front.append("F")

x_train = np.stack(x_train, axis=0) #stacking the data in the form of array
# print(x_train)
y_train_front = np.stack(y_train_front, axis=0)
y_train_back = np.stack(y_train_back, axis=0)

print(x_train.shape)
print(y_train_front.shape)
print(y_train_back.shape)

(218, 660)
(218,)
(218,)


In [8]:
# NN Model Training

x_train_cpy, x_test_cpy, y_train_back_cpy, y_test_back_cpy = train_test_split(x_train, y_train_back, test_size=0.2)

parameters = {
    'hidden_layer_sizes': [(125,64,32,16,8,4,2)], # 8 layers
    'activation': ['relu'], # 'logistic' is slow
    'solver': ['adam'], # 'lbfgs' is slow
    'alpha': [0.01],
    'learning_rate': ['constant'], # 'adaptive' is slow, 'constant' is default
    'max_iter': [5000],
}

# Create MLPClassifier instance
clf = MLPClassifier(random_state=42)

# Use GridSearchCV for parameter tuning
grid_search = GridSearchCV(clf, parameters, cv=5) # 5-fold cross validation 
grid_search.fit(x_train_cpy, y_train_back_cpy)

# Best parameters found
print("Best parameters:", grid_search.best_params_)

# Evaluate the best model
best_model = grid_search.best_estimator_
accuracy = best_model.score(x_test_cpy, y_test_back_cpy)
print("Accuracy of the best model:", accuracy)

Best parameters: {'activation': 'relu', 'alpha': 0.01, 'hidden_layer_sizes': (125, 64, 32, 16, 8, 4, 2), 'learning_rate': 'constant', 'max_iter': 5000, 'solver': 'adam'}
Accuracy of the best model: 0.6363636363636364


In [9]:
# Testing Zone
y_pred = best_model.predict(x_test_cpy)
mse = mean_squared_error(y_test_back_cpy, y_pred)
print("Mean Squared Error:", mse)
print("Predicted:", y_pred)
print("Actual:", y_test_back_cpy)
print("Accuracy:", accuracy)


ValueError: dtype='numeric' is not compatible with arrays of bytes/strings.Convert your data to numeric values explicitly instead.